In [50]:
import pandas as pd
from pymongo import MongoClient
from collections import defaultdict
from urllib.parse import quote_plus
import requests

In [51]:

url = f"https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/6507?date=19750101:20231231"

In [52]:
resp = requests.get(url, params=None)
resp.raise_for_status()
data = resp.json()

In [53]:
df = pd.json_normalize(
    data,
    record_path="Data",
    meta=["COD", "Nombre", "FK_Unidad", "FK_Escala"],
    record_prefix="",       # opcional: no anteponer nada
    meta_prefix=""          # opcional
)

# 3) Convertir la fecha (ms desde 1970) a datetime legible
df["Fecha"] = pd.to_datetime(df["Fecha"], unit="ms")

df.head()

,Fecha,FK_TipoDato,FK_Periodo,Anyo,Valor,Secreto,COD,Nombre,FK_Unidad,FK_Escala
0,2022-12-31 23:00:00,1,28,2023,320656.0,False,MNP162,Total Nacional. Nacimiento. Total. Todas las e...,120,1
1,2021-12-31 23:00:00,1,28,2022,329251.0,False,MNP162,Total Nacional. Nacimiento. Total. Todas las e...,120,1
2,2020-12-31 23:00:00,1,28,2021,337380.0,False,MNP162,Total Nacional. Nacimiento. Total. Todas las e...,120,1
3,2019-12-31 23:00:00,1,28,2020,341315.0,False,MNP162,Total Nacional. Nacimiento. Total. Todas las e...,120,1
4,2018-12-31 23:00:00,1,28,2019,360617.0,False,MNP162,Total Nacional. Nacimiento. Total. Todas las e...,120,1


In [54]:
for columna in df.columns: 
    print(f"Nombre de la columna: {columna}")
    print(df[columna].unique())
    print("----")

Nombre de la columna: Fecha
<DatetimeArray>
['2022-12-31 23:00:00', '2021-12-31 23:00:00', '2020-12-31 23:00:00',
 '2019-12-31 23:00:00', '2018-12-31 23:00:00', '2017-12-31 23:00:00',
 '2016-12-31 23:00:00', '2015-12-31 23:00:00', '2014-12-31 23:00:00',
 '2013-12-31 23:00:00', '2012-12-31 23:00:00', '2011-12-31 23:00:00',
 '2010-12-31 23:00:00', '2009-12-31 23:00:00', '2008-12-31 23:00:00',
 '2007-12-31 23:00:00', '2006-12-31 23:00:00', '2005-12-31 23:00:00',
 '2004-12-31 23:00:00', '2003-12-31 23:00:00', '2002-12-31 23:00:00',
 '2001-12-31 23:00:00', '2000-12-31 23:00:00', '1999-12-31 23:00:00',
 '1998-12-31 23:00:00', '1997-12-31 23:00:00', '1996-12-31 23:00:00',
 '1995-12-31 23:00:00', '1994-12-31 23:00:00', '1993-12-31 23:00:00',
 '1992-12-31 23:00:00', '1991-12-31 23:00:00', '1990-12-31 23:00:00',
 '1989-12-31 23:00:00', '1988-12-31 23:00:00', '1987-12-31 23:00:00',
 '1986-12-31 23:00:00', '1985-12-31 23:00:00', '1984-12-31 23:00:00',
 '1983-12-31 23:00:00', '1982-12-31 23:00:00',

In [55]:
# 1) Extraer Comunidad (todo hasta el primer punto) y Sexo (lo que sigue a "Nacimiento. " hasta el siguiente punto)
df['Comunidad'] = df['Nombre'].str.extract(r'^(.*?)\.')
df['Sexo']       = df['Nombre'].str.extract(r'Nacimiento\. ([^\.]+)\.')

# 2) Seleccionar y renombrar columnas
df2 = df.loc[:, ['Comunidad', 'Sexo', 'Anyo', 'Valor']].copy()
df2.rename(columns={
    'Comunidad': 'Comunidades y Ciudades Autónomas',
    'Anyo':       'Periodo',
    'Valor':      'Total'
}, inplace=True)

# 3) Ordenar por Comunidad y luego por Sexo en el orden Total→Hombres→Mujeres
ordering = {'Total': 0, 'Hombres': 1, 'Mujeres': 2}
df2['sexo_ord'] = df2['Sexo'].map(ordering)
df2.sort_values(['Comunidades y Ciudades Autónomas', 'sexo_ord'], inplace=True)
df2.drop(columns='sexo_ord', inplace=True)

In [56]:
df2.head()

,Comunidades y Ciudades Autónomas,Sexo,Periodo,Total
147,Andalucía,Total,2023,61397.0
148,Andalucía,Total,2022,62967.0
149,Andalucía,Total,2021,65650.0
150,Andalucía,Total,2020,65522.0
151,Andalucía,Total,2019,69397.0


In [57]:
for columna in df2.columns: 
    print(f"Nombre de la columna: {columna}")
    l = df2[columna].unique()
    print(l)
    print(len(l))
    print("----")

Nombre de la columna: Comunidades y Ciudades Autónomas
['Andalucía' 'Aragón' 'Asturias, Principado de' 'Balears, Illes'
 'Canarias' 'Cantabria' 'Castilla - La Mancha' 'Castilla y León'
 'Cataluña' 'Ceuta' 'Comunitat Valenciana' 'Extranjero' 'Extremadura'
 'Galicia' 'Madrid, Comunidad de' 'Melilla' 'Murcia, Región de'
 'Navarra, Comunidad Foral de' 'País Vasco' 'Rioja, La' 'Total Nacional']
21
----
Nombre de la columna: Sexo
['Total' 'Hombres' 'Mujeres']
3
----
Nombre de la columna: Periodo
[2023 2022 2021 2020 2019 2018 2017 2016 2015 2014 2013 2012 2011 2010
 2009 2008 2007 2006 2005 2004 2003 2002 2001 2000 1999 1998 1997 1996
 1995 1994 1993 1992 1991 1990 1989 1988 1987 1986 1985 1984 1983 1982
 1981 1980 1979 1978 1977 1976 1975]
49
----
Nombre de la columna: Total
[ 61397.  62967.  65650. ... 317256. 328224. 322992.]
2880
----


In [58]:
# 1) Extraer Comunidad y Sexo
df['Comunidad'] = (
    df['Nombre']
      .str.extract(r'^(.*?)\.')   # todo hasta el primer punto
      .iloc[:,0]
      .str.strip()
      .replace({'Total Nacional': 'Total'})  # renombrar para tu Excel
)

df['Sexo'] = (
    df['Nombre']
      .str.extract(r'Nacimiento\. ([^\.]+)\.')
      .iloc[:,0]
      .str.strip()
)

# 2) Montar df2 y renombrar columnas
df2 = (
    df.loc[:, ['Comunidad', 'Sexo', 'Anyo', 'Valor']]
      .rename(columns={
          'Comunidad': 'Comunidades y Ciudades Autónomas',
          'Anyo':       'Periodo',
          'Valor':      'Total'
      })
)

# 3) Definir orden para Sexo y Comunidades
sexo_order = ['Total', 'Hombres', 'Mujeres']
df2['Sexo'] = pd.Categorical(df2['Sexo'], categories=sexo_order, ordered=True)

# Para Comunidades: queremos “Total” primero y luego alfabético
comunidades = list(df2['Comunidades y Ciudades Autónomas'].unique())
# nos aseguramos de que “Total” está solo una vez y al principio
comunidades = ['Total'] + sorted([c for c in comunidades if c != 'Total'])
df2['Comunidades y Ciudades Autónomas'] = pd.Categorical(
    df2['Comunidades y Ciudades Autónomas'],
    categories=comunidades,
    ordered=True
)

# 4) Ordenar el DataFrame
df2 = df2.sort_values(
    ['Comunidades y Ciudades Autónomas', 'Sexo', 'Periodo'],
    ignore_index=True
)

In [59]:
df2.head()

,Comunidades y Ciudades Autónomas,Sexo,Periodo,Total
0,Total,Total,1975,669378.0
1,Total,Total,1976,677456.0
2,Total,Total,1977,656357.0
3,Total,Total,1978,636892.0
4,Total,Total,1979,601992.0


In [60]:
for columna in df2.columns: 
    print(f"Nombre de la columna: {columna}")
    l = df2[columna].unique()
    print(len(l))
    print("----")

Nombre de la columna: Comunidades y Ciudades Autónomas
21
----
Nombre de la columna: Sexo
3
----
Nombre de la columna: Periodo
49
----
Nombre de la columna: Total
2880
----


In [61]:
df2.head(49)

,Comunidades y Ciudades Autónomas,Sexo,Periodo,Total
0,Total,Total,1975,669378.0
1,Total,Total,1976,677456.0
2,Total,Total,1977,656357.0
3,Total,Total,1978,636892.0
4,Total,Total,1979,601992.0
5,Total,Total,1980,571018.0
6,Total,Total,1981,533008.0
7,Total,Total,1982,515706.0
8,Total,Total,1983,485352.0
9,Total,Total,1984,473281.0


In [62]:
df_final= df2.copy()

In [63]:
df_final["sex"] = df_final["Sexo"].map({"Hombres": "M", "Mujeres": "F", "Total": "T"})

In [64]:
df_final.rename(columns={"Total": "births", "Periodo": "year"}, inplace=True)

In [65]:
df_final.sample()

,Comunidades y Ciudades Autónomas,Sexo,year,births,sex
2085,Galicia,Total,2002,19327.0,T


In [66]:
print(list(df_final['Comunidades y Ciudades Autónomas'].unique()))

['Total', 'Andalucía', 'Aragón', 'Asturias, Principado de', 'Balears, Illes', 'Canarias', 'Cantabria', 'Castilla - La Mancha', 'Castilla y León', 'Cataluña', 'Ceuta', 'Comunitat Valenciana', 'Extranjero', 'Extremadura', 'Galicia', 'Madrid, Comunidad de', 'Melilla', 'Murcia, Región de', 'Navarra, Comunidad Foral de', 'País Vasco', 'Rioja, La']


In [67]:
df_codigos_ccaa_ine = pd.read_csv('../data/codigos_ine_prov_ccaa/codigos_ccaa.csv', sep=';', dtype={
        'cod': 'string', 
        'nombre_ine': 'string',
    })
df_codigos_ccaa_mongo = pd.read_csv('../data/codigos_ine_prov_ccaa/codigos_ccaa_sin_CeutaMelilla.csv', sep=';', dtype={
        'cod_ccaa': 'string', 
        'name': 'string',
    })

In [68]:
# Mapear nombre -> código INE
# Creamos un dict de mapeo a partir de df_codigos_ccaa_ine
map_name_to_code = dict(
    zip(df_codigos_ccaa_ine['nombre_ine'], df_codigos_ccaa_ine['cod'])
)

In [69]:
map_name_to_code['Total'] = "Total"
map_name_to_code['Extranjero'] = "Extranjero"

In [70]:
# Aplicamos el mapeo para crear la columna code_ine_ccaa en df_final
df_final['code_ine_ccaa'] = df_final['Comunidades y Ciudades Autónomas'].map(map_name_to_code)

# Comprueba que no haya NaN inesperados
missing = df_final[df_final['code_ine_ccaa'].isna()]['Comunidades y Ciudades Autónomas'].unique()
if len(missing):
    print("No se han encontrado códigos para estos nombres:", missing)

In [71]:
list(df_final['code_ine_ccaa'].unique())

['Total',
 '01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '08',
 '07',
 '09',
 '18',
 '10',
 'Extranjero',
 '11',
 '12',
 '13',
 '19',
 '14',
 '15',
 '16',
 '17']

In [72]:
# Mapear código INE → nombre para Metabase
# Creamos un dict de mapeo a partir de df_codigos_ccaa_mongo
map_code_to_mongo_name = dict(
    zip(df_codigos_ccaa_mongo['cod_ccaa'], df_codigos_ccaa_mongo['name'])
)

In [73]:
map_code_to_mongo_name["Total"] = "Total"
map_code_to_mongo_name["18"] = "Ceuta"
map_code_to_mongo_name["19"] = "Melilla"
map_code_to_mongo_name["Extranjero"] = "Extranjero"

In [74]:
# Aplicamos el mapeo para crear la columna name_ccaa
df_final['name_ccaa'] = df_final['code_ine_ccaa'] \
    .map(map_code_to_mongo_name)

# Verifica ausencias
missing2 = df_final[df_final['name_ccaa'].isna()]['code_ine_ccaa'].unique()
if len(missing2):
    print("Estos códigos INE no tienen nombre en tu tabla Mongo:", missing2)

In [75]:
# 3) Seleccionar solo las columnas definitivas
df_clean = df_final[[
    'year',
    'sex',
    'births',
    'code_ine_ccaa',
    'name_ccaa'
]].copy()

# Comprobación final
print("Estructura final:")
print(df_clean.dtypes)
print(df_clean.head())

Estructura final:
year                int64
sex              category
births            float64
code_ine_ccaa    category
name_ccaa        category
dtype: object
   year sex    births code_ine_ccaa name_ccaa
0  1975   T  669378.0         Total     Total
1  1976   T  677456.0         Total     Total
2  1977   T  656357.0         Total     Total
3  1978   T  636892.0         Total     Total
4  1979   T  601992.0         Total     Total


In [76]:
# Configurar conexión
usuario = "jalope"
contrasena = "admin"
host = "127.0.0.1"
puerto = "27250"

uri = f"mongodb://{quote_plus(usuario)}:{quote_plus(contrasena)}@{host}:{puerto}/?directConnection=true"
client = MongoClient(uri)
db = client["tfm_db"]

In [77]:
coll = db['INE_6507_POBLACION_NACIMIENTOS_RAW']
coll.drop()
coll.insert_many(df_clean.to_dict('records'))
print("Registros en la colección: ", coll.count_documents({}))
print("Número de filas de df_raw: ", len(df_clean))

Registros en la colección:  3087
Número de filas de df_raw:  3087
